In [2]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

SKIP_NEGATIVES = True
NEGATIVE_CLASS = "unrelated_image"

def xml_to_csv(path, skipNegatives):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        if root.find('object'):
            for member in root.findall('object'):
                bbx = member.find('bndbox')                
                xmin = round(float(bbx.find('xmin').text))
                ymin = round(float(bbx.find('ymin').text))
                xmax = round(float(bbx.find('xmax').text))
                ymax = round(float(bbx.find('ymax').text))
                label = member.find('name').text
                value = (root.find('filename').text,
                        int(root.find('size')[0].text),
                        int(root.find('size')[1].text),
                        label,
                        xmin,
                        ymin,
                        xmax,
                        ymax
                        )
                print(value)
                xml_list.append(value)
        elif not skipNegatives:
            value = (root.find('filename').text,
                        int(root.find('size')[0].text),
                        int(root.find('size')[1].text),
                        NEGATIVE_CLASS,
                        0,
                        0,
                        0,
                        0
                        )
            print(value)
            xml_list.append(value)

    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    datasets = ['train', 'validation']

    for ds in datasets:
        image_path = os.path.join(os.getcwd(), 'Images', ds)
        xml_df = xml_to_csv(image_path, SKIP_NEGATIVES)
        xml_df.to_csv('Data/{}_data.csv'.format(ds), index=None)
        print('Successfully converted xml to csv.')


main()

('Blast (1).JPG', 3381, 3448, 'Blast', 393, 582, 1091, 1582)
('Blast (1).JPG', 3381, 3448, 'Blast', 1376, 319, 1867, 1845)
('Blast (1).JPG', 3381, 3448, 'Blast', 2402, 60, 3285, 1125)
('Blast (1).JPG', 3381, 3448, 'Blast', 1264, 2250, 2677, 3392)
('Blast (1).JPG', 3381, 3448, 'Blast', 1221, 17, 1488, 763)
('Blast (1).JPG', 3381, 3448, 'Blast', 2005, 1224, 2255, 1827)
('Blast (1).JPG', 3381, 3448, 'Blast', 406, 2138, 656, 2504)
('Blast (10).JPG', 3455, 4855, 'Blast', 1643, 218, 1885, 558)
('Blast (10).JPG', 3455, 4855, 'Blast', 1357, 874, 1837, 1991)
('Blast (10).JPG', 3455, 4855, 'Blast', 926, 2330, 1552, 4855)
('Blast (2).JPG', 2271, 3375, 'Blast', 617, 1734, 1009, 2223)
('Blast (3).JPG', 2307, 4070, 'Blast', 512, 1292, 940, 1694)
('Blast (3).JPG', 2307, 4070, 'Blast', 1886, 61, 2263, 417)
('Blast (3).JPG', 2307, 4070, 'Blast', 1683, 509, 1947, 860)
('Blast (3).JPG', 2307, 4070, 'Blast', 884, 646, 1118, 850)
('Blast (4).JPG', 5184, 3456, 'Blast', 3862, 2199, 4752, 2462)
('Blast (4).JP

In [2]:
from tensorflow import keras
import pandas as pd
import os

TRAINING_CSV_FILE = 'Data/train_data.csv'
TRAINING_IMAGE_DIR = 'Images/train'

training_image_records = pd.read_csv(TRAINING_CSV_FILE)

train_image_path = os.path.join(os.getcwd(), TRAINING_IMAGE_DIR)

train_images = []
train_targets = []
train_labels = []
classes = ["Blast", "False_Smut", "RiceBug", "SheathRot", "StemBorer", "unrelated_image"]

for index, row in training_image_records.iterrows():
    
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row
    
    train_image_fullpath = os.path.join(train_image_path, filename)
    train_img = keras.preprocessing.image.load_img(train_image_fullpath, target_size=(int(height/10), int(width/10)))
    train_img_arr = keras.preprocessing.image.img_to_array(train_img)
    
    
    xmin = round(int(xmin/10)/ int(width/10), 2)
    ymin = round(int(ymin/10)/ int(height/10), 2)
    xmax = round(int(xmax/10)/ int(width/10), 2)
    ymax = round(int(ymax/10)/ int(height/10), 2)
    
    train_images.append(train_img_arr)
    train_targets.append((xmin, ymin, xmax, ymax))
    train_labels.append(classes.index(class_name))

In [2]:
from tensorflow import keras
VALIDATION_CSV_FILE = 'Data/validation_data.csv'
VALIDATION_IMAGE_DIR = 'Images/validation'

validation_image_records = pd.read_csv(VALIDATION_CSV_FILE)

validation_image_path = os.path.join(os.getcwd(), VALIDATION_IMAGE_DIR)

validation_images = []
validation_targets = []
validation_labels = []
classes = ["Blast", "False_Smut", "RiceBug", "SheathRot", "StemBorer", "unrelated_image"]

for index, row in validation_image_records.iterrows():
    
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row
    
    validation_image_fullpath = os.path.join(validation_image_path, filename)
    validation_img = keras.preprocessing.image.load_img(validation_image_fullpath, target_size=(int(height/10), int(width/10)))
    validation_img_arr = keras.preprocessing.image.img_to_array(validation_img)
    
    
    xmin = round(int(xmin/10)/ int(width/10), 2)
    ymin = round(int(ymin/10)/ int(height/10), 2)
    xmax = round(int(xmax/10)/ int(width/10), 2)
    ymax = round(int(ymax/10)/ int(height/10), 2)
    
    validation_images.append(validation_img_arr)
    validation_targets.append((xmin, ymin, xmax, ymax))
    validation_labels.append(classes.index(class_name))

In [3]:
import numpy as np

train_images = np.array(train_images) / 255.0
train_targets = np.array(train_targets, dtype="float32")
train_labels = np.array(train_labels)
validation_images = np.array(validation_images) / 255.0
validation_targets = np.array(validation_targets, dtype="float32")
validation_labels = np.array(validation_labels)

In [4]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

In [5]:
IMAGE_SIZE = [299, 299]
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
for layer in inception.layers:
    layer.trainable = False
flatten = Flatten()(inception.output)
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid", name="bounding_box")(bboxHead)
softmaxHead = Dense(6, activation="softmax", name="class_label")(flatten)
model = Model(inputs=inception.input, outputs=(bboxHead, softmaxHead))

In [7]:
from tensorflow.keras.optimizers import Adam

INIT_LR = 1e-4
NUM_EPOCHS = 1
BATCH_SIZE = 32
losses = {
            "class_label": "categorical_crossentropy",
            "bounding_box": "mean_squared_error"
         }
lossWeights = {
            "class_label": 1.0,
            "bounding_box": 1.0
}
opt = Adam(lr=INIT_LR)
model.compile(
              loss=losses,
              optimizer=opt,
              metrics=['accuracy'], 
              loss_weights=lossWeights
             )
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [8]:
trainTargets = {
    "class_label": train_labels,
    "bounding_box": train_targets
}
validationTargets = {
    "class_label": validation_labels,
    "bounding_box": validation_targets
}

In [9]:
H = model.fit(train_images, trainTargets, 
              validation_data=(validation_images, validationTargets), 
              batch_size=BATCH_SIZE, 
              epochs=NUM_EPOCHS,
              verbose=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).